In [1]:
import numpy as np
import pandas as pd
import os, sys, json
import twint
import nest_asyncio

In [2]:
nest_asyncio.apply()

datapath = '../data/twitter_data'

In [3]:
def tweets2df(tweets, country):
    def tweet2json(tweet):
        return {
            'id': tweet.id,
            'datestamp': tweet.datestamp,
            'timestamp': tweet.timestamp,
            'username': tweet.username,
            'tweet': tweet.tweet,
            'replies_count': tweet.replies_count,
            'retweets_count': tweet.retweets_count,
            'likes_count': tweet.likes_count,
            'url': tweet.link,
            'country': country
        }
    
    jsons = [tweet2json(tweet) for tweet in tweets]
    df = pd.DataFrame(jsons)
    return df

In [4]:
def config_twint(username, since='2020-01-22'):
    twint.output.tweets_list = []
    tw = twint.Config()
    tw.Username = newspaper
    tw.Since = '2020-01-22'
    tw.Store_object = True
    tw.Stats = True
    tw.Hide_output = True
    return tw

In [9]:
def create(country, newspaper):
    """
    """
    print(f'Creating {newspaper} of {country}')
    dfname = f'{country}_{newspaper}.tsv'
    tw = config_twint(newspaper)

    twint.run.Search(tw)

    df = tweets2df(twint.output.tweets_list, country)
    if (dfname == 'China_chinaorgcn.tsv'):
        df.to_json(f'{datapath}/{dfname}')
    else:
        df.to_csv(f'{datapath}/{dfname}', sep='\t', index=False)

def update(country, newspaper):
    """
    """
    print(f'Updating {newspaper} of {country}')
    dfname = f'{country}_{newspaper}.tsv'
    past = pd.read_csv(f'{datapath}/{dfname}', sep='\t', engine='python')
    most_recent = past['datestamp'][0] + " " + past['timestamp'][0] 
    
    tw = config_twint(newspaper, most_recent)
    twint.run.Search(tw)

    df = tweets2df(twint.output.tweets_list, country)
    df = pd.concat([df, past], ignore_index=True)
    df.to_csv(f'{datapath}/{dfname}', sep='\t', index=False)

In [6]:
countries = {
    'US': ['cnn', 'nytimes', 'huffpost', 'FoxNews'],
    'Brazil': ['g1', 'folha', 'Estadao', 'JornalOGlobo', 'JornaldoBrasil'],
    'Italy': ['repubblica', 'Corriere', 'Libero_official', 'virgilio_it'],
    'UK': ['BBCNews', 'guardian', 'MailOnline', 'Telegraph'],
    'China': ['ChinaDaily', 'PDChina', 'shanghaidaily', 'chinaorgcn']
}

In [7]:
fields = ['id', 'datestamp, timestamp', 'username', 'tweet', 'replies_count', 'retweets_count', 'likes_count', 'url']
formatted = '"' + "\t".join(['{' + field + '}' for field in fields]) + '"'

In [10]:
for country in countries.keys():
    if (country != 'China'): continue
    newspapers = countries[country]
    for newspaper in newspapers:
        dfname = f'{country}_{newspaper}.tsv'
        if dfname in os.listdir(datapath) and False:
            update(country, newspaper)
        else:
            create(country, newspaper)
    
print('done')

Creating ChinaDaily of China
Creating PDChina of China
Creating shanghaidaily of China
Creating chinaorgcn of China
done


In [ ]:
"""
for country in countries.keys():
    newspapers = countries[country]
    for newspaper in newspapers:
        dfname = f'{country}_{newspaper}.tsv'
        df = pd.read_csv(f'{datapath}/{dfname}', sep='\t', engine='python')
        df.drop_duplicates(subset='id', keep=False, inplace=True)
        df.to_csv(f'{datapath}/{dfname}', sep='\t', index=False)
"""

In [ ]:
df = tweets2df(twint.output.tweets_list, country)
df.to_json(f'{datapath}/{dfname}')

In [ ]:
df.head()

In [ ]:
dfname